# 第13章 计算机视觉 

## 13.1 图像增广

### 练习13.1.1

在不使用图像增广的情况下训练模型：`train_with_data_aug(no_aug, no_aug)`。比较使用和不使用图像增广的训练结果和测试精度。这个对比实验能支持图像增广可以减轻过拟合的论点吗？为什么？

**解答：**

### 练习13.1.2

在基于CIFAR-10数据集的模型训练中结合多种不同的图像增广方法。它能提高测试准确性吗？

**解答：**

### 练习13.1.3

参阅深度学习框架的在线文档。它还提供了哪些其他的图像增广方法？

**解答：**

## 13.2 微调

### 练习13.2.1

继续提高`finetune_net`的学习率，模型的准确性如何变化？

**解答：**

### 练习13.2.2

在比较实验中进一步调整`finetune_net`和`scratch_net`的超参数。它们的准确性还有不同吗？

**解答：**

### 练习13.2.3

将输出层`finetune_net`之前的参数设置为源模型的参数，在训练期间不要更新它们。模型的准确性如何变化？提示：可以使用以下代码。

```python
for param in finetune_net.parameters():
    param.requires_grad = False
```

**解答：**

### 练习13.2.4

事实上，`ImageNet`数据集中有一个“热狗”类别。我们可以通过以下代码获取其输出层中的相应权重参数，但是我们怎样才能利用这个权重参数？

```python
weight = pretrained_net.fc.weight
hotdog_w = torch.split(weight.data, 1, dim=0)[934]
hotdog_w.shape
```

**解答：**

## 13.3 目标检测和边界框

### 练习13.3.1

找到另一张图像，然后尝试标记包含该对象的边界框。比较标注边界框和标注类别哪个需要更长的时间？

**解答：**

### 练习13.3.2

为什么`box_corner_to_center`和`box_center_to_corner`的输入参数的最内层维度总是4？

**解答：**

## 13.4 锚框

### 练习13.4.1

在`multibox_prior`函数中更改`sizes`和`ratios`的值。生成的锚框有什么变化？

**解答：**

### 练习13.4.2

构建并可视化两个IoU为0.5的边界框。它们是怎样重叠的？

**解答：**

### 练习13.4.3

在13.4.3节和13.4.4节中修改变量`anchors`，结果如何变化？

**解答：**

### 练习13.4.4                
                
非极大值抑制是一种贪心算法，它通过*移除*来抑制预测的边界框。是否存在一种可能，被移除的一些框实际上是有用的？如何修改这个算法来柔和地抑制？可以参考Soft-NMS。

**解答：**

### 练习13.4.5
        
如果非手动，非最大限度的抑制可以被学习吗？

**解答：**

## 13.5 多尺度目标检测

### 练习13.5.1

根据我们在7.1节中的讨论，深度神经网络学习图像特征级别抽象层次，随网络深度的增加而升级。在多尺度目标检测中，不同尺度的特征映射是否对应于不同的抽象层次？为什么？

**解答：**

### 练习13.5.2

在13.5.1节中的实验里的第一个尺度（`fmap_w=4, fmap_h=4`）下，生成可能重叠的均匀分布的锚框。

**解答：**

### 练习13.5.3

给定形状为$1 \times c \times h \times w$的特征图变量，其中$c$、$h$和$w$分别是特征图的通道数、高度和宽度。怎样才能将这个变量转换为锚框类别和偏移量？输出的形状是什么？

**解答：**

## 13.6 目标检测数据集

### 练习13.6.1

在香蕉检测数据集中演示其他带有真实边界框的图像。它们在边界框和目标方面有什么不同？

**解答：**

### 练习13.6.2

假设我们想要将数据增强（例如随机裁剪）应用于目标检测。它与图像分类中的有什么不同？提示：如果裁剪的图像只包含物体的一小部分会怎样？

**解答：**

## 13.7 单发多框检测（SSD）

### 练习13.7.1

能通过改进损失函数来改进单发多框检测吗？例如，将预测偏移量用到的$L_1$范数损失替换为平滑$L_1$范数损失。它在零点附近使用平方函数从而更加平滑，这是通过一个超参数$\sigma$来控制平滑区域的：

$$
f(x) =
    \begin{cases}
    (\sigma x)^2/2,& \text{if }|x| < 1/\sigma^2\\
    |x|-0.5/\sigma^2,& \text{otherwise}
    \end{cases}
$$

当$\sigma$非常大时，这种损失类似于$L_1$范数损失。当它的值较小时，损失函数较平滑。

In [ ]:
def smooth_l1(data, scalar):
    out = []
    for i in data:
        if abs(i) < 1 / (scalar ** 2):
            out.append(((scalar * i) ** 2) / 2)
        else:
            out.append(abs(i) - 0.5 / (scalar ** 2))
    return torch.tensor(out)

sigmas = [10, 1, 0.5]
lines = ['-', '--', '-.']
x = torch.arange(-2, 2, 0.1)
d2l.set_figsize()

for l, s in zip(lines, sigmas):
    y = smooth_l1(x, scalar=s)
    d2l.plt.plot(x, y, l, label='sigma=%.1f' % s)
d2l.plt.legend();

此外，在类别预测时，实验中使用了交叉熵损失：设真实类别$j$的预测概率是$p_j$，交叉熵损失为$-\log p_j$。我们还可以使用焦点损失。给定超参数$\gamma > 0$和$\alpha > 0$，此损失的定义为：

$$ - \alpha (1-p_j)^{\gamma} \log p_j.$$

可以看到，增大$\gamma$可以有效地减少正类预测概率较大时（例如$p_j > 0.5$）的相对损失，因此训练可以更集中在那些错误分类的困难示例上。

In [ ]:
def focal_loss(gamma, x):
    return -(1 - x) ** gamma * torch.log(x)

x = torch.arange(0.01, 1, 0.01)
for l, gamma in zip(lines, [0, 1, 5]):
    y = d2l.plt.plot(x, focal_loss(gamma, x), l, label='gamma=%.1f' % gamma)
d2l.plt.legend();

**解答：**

### 练习13.7.2

由于篇幅限制，我们在本节中省略了单发多框检测模型的一些实现细节。能否从以下几个方面进一步改进模型：

1. 当目标比图像小得多时，模型可以将输入图像调大；
2. 通常会存在大量的负锚框。为了使类别分布更加平衡，我们可以将负锚框的高和宽减半；
3. 在损失函数中，给类别损失和偏移损失设置不同比重的超参数；
4. 使用其他方法评估目标检测模型，例如单发多框检测论文 :cite:`Liu.Anguelov.Erhan.ea.2016`中的方法。

**解答：**

## 13.8 区域卷积神经网络（R-CNN）系列

### 练习13.8.1

我们能否将目标检测视为回归问题（例如预测边界框和类别的概率）？可以参考YOLO模型的设计。

**解答：**

### 练习13.8.2        
        
将单发多框检测与本节介绍的方法进行比较。他们的主要区别是什么？可以参考参考文献[195]中的图2。

**解答：**

## 13.9 语义分割和数据集

### 练习13.9.1

如何在自动驾驶和医疗图像诊断中应用语义分割？还能想到其他领域的应用吗？

**解答：**

**解答：**### 练习13.9.2

回想一下13.1节中对数据增强的描述。图像分类中使用的哪种图像增强方法是难以用于语义分割的？

**解答：**

## 13.10 转置卷积

### 练习13.10.1

在13.10.3节中，卷积输入`X`和转置的卷积输出`Z`具有相同的形状。他们的数值也相同吗？为什么？

**解答：**

### 练习13.10.2
        
1. 使用矩阵乘法来实现卷积是否有效率？为什么？

**解答：**

## 13.11 全卷积网络

### 练习13.11.1

如果将转置卷积层改用Xavier随机初始化，结果有什么变化？

**解答：**

### 练习13.11.2

调节超参数，能进一步提升模型的精度吗？

**解答：**

### 练习13.11.3

预测测试图像中所有像素的类别。

**解答：**

### 练习13.11.4

最初的全卷积网络的论文中还使用了某些卷积神经网络中间层的输出。试着实现这个想法。

**解答：**

## 13.12 风格迁移

### 练习13.12.1

选择不同的内容和风格层，输出有什么变化？

**解答：**

### 练习13.12.2

调整损失函数中的权重超参数。输出是否保留更多内容或减少更多噪点？

**解答：**

### 练习13.12.3

替换实验中的内容图像和风格图像，能创作出更有趣的合成图像吗？

**解答：**

### 练习13.12.4

我们可以对文本使用风格迁移吗？提示:可以参阅调查报告[68]。

**解答：**

## 13.13 实战 Kaggle 比赛：图像分类 (CIFAR-10) 

### 练习13.13.1

在这场Kaggle竞赛中使用完整的CIFAR-10数据集。将超参数设为`batch_size = 128`，`num_epochs = 100`，`lr = 0.1`，`lr_period = 50`，`lr_decay = 0.1`。看看在这场比赛中能达到什么准确度和排名。能进一步改进吗？

**解答：**

### 练习13.13.2

不使用图像增广时，能获得怎样的准确度？

**解答：**

## 13.14 实战Kaggle比赛：狗的品种识别（ImageNet Dogs）

### 练习13.14.1

试试使用完整Kaggle比赛数据集，增加`batch_size`（批量大小）和`num_epochs`（迭代轮数），或者设计其它超参数为`lr = 0.01`，`lr_period = 10`，和`lr_decay = 0.1`时，能取得什么结果？

**解答：**

### 练习13.14.2

如果使用更深的预训练模型，会得到更好的结果吗？如何调整超参数？能进一步改善结果吗？

**解答：**